# **HW13 Meta Learning: Omniglot Few-Shot Classification**

## **Step 1: Download the Dataset**
- Download the Omniglot Dataset.
- Data Augmentation has been done on the dataset.

In [1]:
workspace_dir = '.'
!gdown --id 1FLDrQ0k-iJ-mk8ors0WItqvwgu0w9J0U --output "{workspace_dir}/Omniglot.tar.gz"

Downloading...
From: https://drive.google.com/uc?id=1FLDrQ0k-iJ-mk8ors0WItqvwgu0w9J0U
To: /content/Omniglot.tar.gz
26.4MB [00:00, 84.1MB/s]


- Unzip all the images from the dataset.

In [2]:
# unzip all the images in the dataset
!tar -zxf "{workspace_dir}/Omniglot.tar.gz" -C "{workspace_dir}/" # use "-zxf" won't print out all the image names

- Take a look at what the images in Omniglot dataset look like.

In [3]:
from PIL import Image
from IPython.display import display
for i in range(10, 20):
  image = Image.open("Omniglot/images_background/Japanese_(hiragana).0/character13/0500_" + str (i) + ".png")
  display(image)

## **Step 2: Construct the Model**
- In step 2, we'll build the model.
- Let's start with importing all the required modules.

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import glob
from tqdm import tqdm
import numpy as np
from collections import OrderedDict

- Let's look at the MAML algorithm before we constructing our model.
<p align="center"><img width="70%" src="https://i.imgur.com/AJlQ5ah.jpg"></p>
- At step 10 (inner loop), the `functional_forward` method is used to compute the output logits of the input image as **we are computing the gradient on θ instead of θ'**.
- The `forward` method in `nn.Module` is computing the gradient on θ'.

In [5]:
def ConvBlock(in_ch, out_ch):
  return nn.Sequential(
      nn.Conv2d(in_ch, out_ch, 3, padding=1),
      nn.BatchNorm2d(out_ch),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2, stride=2)
  )

def ConvBlockFunction(x, w, b, w_bn, b_bn):
  """
  Args:
    x: input tensor of shape (mini-batch, in_channels, iH, iW)
    w: filters of shape (out_channels, in_channels/groups, kH, kW)
    b: optional bias tensor of shape (out_channels)
    w_bn: batch normalization weights
    b_bn: batch normalization bias
  """
  x = F.conv2d(x, w, b, padding=1)
  x = F.batch_norm(x, running_mean=None, running_var=None, weight=w_bn, bias=b_bn, training=True)
  x = F.relu(x)
  x = F.max_pool2d(x, kernel_size=2, stride=2)
  return x

class Classifier(nn.Module):
  def __init__(self, in_ch, n_way):
    super(Classifier, self).__init__()
    self.conv1 = ConvBlock(in_ch, 64)
    self.conv2 = ConvBlock(64, 64)
    self.conv3 = ConvBlock(64, 64)
    self.conv4 = ConvBlock(64, 64)
    self.logits = nn.Linear(64, n_way)
  
  def forward(self, x):
    x = self.conv1(x)
    x = self.conv2(x)
    x = self.conv3(x)
    x = self.conv4(x)
    x = x.view(x.shape[0], -1)
    x = self.logits(x)
    return x

  def functional_forward(self, x, params):
    """
    Args:
      x: input images of shape (batch_size, 1, 28, 28)
      params: the parameters of the model. i.e. the weights & bias of convolution & batch normalization.
            This is a OrderedDict
    """
    for block in [1, 2, 3, 4]:
      x = ConvBlockFunction(
          x,
          params[f'conv{block}.0.weight'],
          params[f'conv{block}.0.bias'],
          params.get(f'conv{block}.1.weight'),
          params.get(f'conv{block}.1.bias')
      )
    x = x.view(x.shape[0], -1)
    x = F.linear(x, params['logits.weight'], params['logits.bias'])
    return x

- In a "n_way", "k_shot" few-shot classification problem, each task will have "n_way" classes. In each class, there will be "k_shot" images.
- The following function is used to generate labels.

In [6]:
def create_label(n_way, k_shot):
  return torch.arange(n_way).repeat_interleave(k_shot).long()

- Let's create a 5-way, 2-shot label

In [7]:
create_label(5, 2)

tensor([0, 0, 1, 1, 2, 2, 3, 3, 4, 4])

- The algorithm in the following is the core of MAML.
- The parameters are updated by "**each meta-batch data**".
- Here, it's a 2nd-order MAML (`inner_train_step=1`).

In [8]:
def MAML(model, optimizer, x, n_way, k_shot, q_query, loss_fn, inner_train_step=1, inner_lr=0.4, train=True):
  """
  Args:
    x: the Omniglot images input for a "meta_step", shape = [batch_size, n_way * (k_shot + q_query), 1, 28, 28]
    n_way: the # of class in each classification task
    k_shot: the # of images in a class when "training"
    q_query: the # of images to update in each class when "testing"
  """
  criterion = loss_fn
  task_loss = [] # for storing the loss for "each task"
  task_acc = [] # for storing the accuracy for "each task"

  for meta_batch in x: # iterate through all tasks
    train_set = meta_batch[:n_way*k_shot] # for update the params in inner loop
    val_set = meta_batch[n_way*k_shot:] # for update the params in outer loop

    # use `fast_weights` to store new parameters θ' in the inner loop
    # since we can NOT modify θ
    fast_weights = OrderedDict(model.named_parameters())

    # the for-loop equals to step 6~7 in algorithm 2
    # we only update the gradients in the inner loop once
    # while some other tasks require updating θ' for several times
    for inner_step in range(inner_train_step):
      train_labels = create_label(n_way, k_shot).cuda()
      logits = model.functional_forward(train_set, fast_weights) # step 6
      loss = criterion(logits, train_labels)  # step 6
      grads = torch.autograd.grad(loss, fast_weights.values(), create_graph=True) # compute the gradient of loss to θ(∇loss), step 6
      fast_weights = OrderedDict(
          (name, param - inner_lr * grad) for ((name, param), grad) in zip(fast_weights.items(), grads)
      ) # update θ -> θ', step 7

    val_label = create_label(n_way, q_query).cuda()
    logits = model.functional_forward(val_set, fast_weights) # compute the logit by `val_set` and `θ'`
    loss = criterion(logits, val_label) # compute the loss by `val_label` and `logits`
    task_loss.append(loss)
    acc = np.asarray([torch.argmax(logits, -1).cpu().numpy() == val_label.cpu().numpy()]).mean()
    task_acc.append(acc)
  
  model.train()
  optimizer.zero_grad()
  meta_batch_loss = torch.stack(task_loss).mean() # update θ by a batch of tasks, step 10
  if train:
    meta_batch_loss.backward()
    optimizer.step()
  task_acc = np.mean(task_acc)
  return meta_batch_loss, task_acc

### **Define the Dataset**
- The dataset will return a image of certain character.
- "k_shot+q_query" images in total.
- The reutrned image size is [k_shot+q_query, 1, 28, 28].

In [9]:
class Omniglot(Dataset):
  def __init__(self, data_dir, k_shot, q_query):
    self.file_list = [f for f in glob.glob(data_dir + "**/character*", recursive=True)]
    self.transform = transforms.Compose([transforms.ToTensor()])
    self.n = k_shot + q_query

  def __getitem__(self, idx):
    sample = np.arange(20)
    np.random.shuffle(sample) # for sampling the characters randomly
    img_path = self.file_list[idx]
    img_list = [f for f in glob.glob(img_path + "**/*.png", recursive=True)]
    img_list.sort()
    imgs = [self.transform(Image.open(img_file)) for img_file in img_list]
    imgs = torch.stack(imgs)[sample[:self.n]] # get "k_shot + q_query" images for each character
    return imgs
  
  def __len__(self):
    return len(self.file_list)

## **Step 3: Start Training**

### **Define the hyperparameters**

In [10]:
n_way = 5
k_shot = 1
q_query = 1
inner_train_step = 1
inner_lr = 0.4
meta_lr = 0.001
meta_batch_size = 32
max_epoch = 40
eval_batches = 20
test_batches = 20
train_data_path = './Omniglot/images_background/'
test_data_path = './Omniglot/images_evaluation/'

### **Define the DataLoader**

In [11]:
train_set, val_set = torch.utils.data.random_split(Omniglot(train_data_path, k_shot, q_query), [3200, 656])
train_loader = DataLoader(
    train_set,
    batch_size = n_way, # the classes in a task. It's not the `meta_batch_size`
    num_workers = 8,
    shuffle = True,
    drop_last = True
)

val_loader = DataLoader(
    val_set,
    batch_size = n_way, # the classes in a task. It's not the `meta_batch_size`
    num_workers = 8,
    shuffle = True,
    drop_last = True
)

test_loader = DataLoader(
    Omniglot(test_data_path, k_shot, q_query),
    batch_size = n_way, # the classes in a task. It's not the `meta_batch_size`
    num_workers = 8,
    shuffle = True,
    drop_last = True

)

train_iter = iter(train_loader)
val_iter = iter(val_loader)
test_iter = iter(test_loader)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


### **Initialize the model optimizer**

In [12]:
meta_model = Classifier(1, n_way).cuda()
optimizer = torch.optim.Adam(meta_model.parameters(), lr=meta_lr)
loss_fn = nn.CrossEntropyLoss().cuda()

### **Define Getting the Meta-batch of data**
- Define a function to get meta batch of data.

In [13]:
def get_meta_batch(meta_batch_size, k_shot, q_query, data_loader, iterator):
  data = []
  for _ in range(meta_batch_size):
    try:
      task_data = iterator.next() # a `task_data` is the data in a single task of shape [n_way, k_shot + q_query, 28, 28]
    except StopIteration:
      iterator = iter(data_loader)
      task_data = iterator.next()
    train_data = task_data[:, :k_shot].reshape(-1, 1, 28, 28)
    val_data = task_data[:, k_shot:].reshape(-1, 1, 28, 28)
    task_data = torch.cat((train_data, val_data), 0)
    data.append(task_data)
  return torch.stack(data).cuda(), iterator

### **Start Training**

In [14]:
for epoch in range(max_epoch):
  print("Epoch %d" %(epoch))
  train_meta_loss = []
  train_acc = []
  for step in tqdm(range(len(train_loader) // (meta_batch_size))): # the step is meta-gradient update for each step
    x, train_iter = get_meta_batch(meta_batch_size, k_shot, q_query, train_loader, train_iter)
    meta_loss, acc = MAML(meta_model, optimizer, x, n_way, k_shot, q_query, loss_fn)
    train_meta_loss.append(meta_loss.item())
    train_acc.append(acc)
  print("Loss: ", np.mean(train_meta_loss))
  print("Accuracy: ", np.mean(train_acc))

  # check the val accuracy
  val_acc = []
  for eval_step in tqdm(range(len(val_loader) // (eval_batches))):
    x, val_iter = get_meta_batch(eval_batches, k_shot, q_query, val_loader, val_iter)
    _, acc = MAML(meta_model, optimizer, x, n_way, k_shot, q_query, loss_fn, inner_train_step=3, train=False) # update 3 inner_step in val
    val_acc.append(acc)
    print("Validation Accuracy: ", np.mean(val_acc))

  0%|          | 0/20 [00:00<?, ?it/s]

Epoch 0


  0%|          | 0/6 [00:00<?, ?it/s]

Loss:  1.5416001558303833
Accuracy:  0.4462499999999999


 17%|█▋        | 1/6 [00:00<00:02,  2.06it/s]

...

Validation Accuracy:  0.9675


 83%|████████▎ | 5/6 [00:02<00:00,  1.85it/s]

Validation Accuracy:  0.9700000000000001


100%|██████████| 6/6 [00:03<00:00,  1.78it/s]

Validation Accuracy:  0.9716666666666668


- Compute the test accuracy.

In [15]:
test_acc = []
for test_step in tqdm(range(len(test_loader) // test_batches)):
  x, test_iter = get_meta_batch(test_batches, k_shot, q_query, test_loader, test_iter)
  _, acc = MAML(meta_model, optimizer, x, n_way, k_shot, q_query, loss_fn, inner_train_step=3, train=False)
  test_acc.append(acc)
print("Testing Accuracy: ", np.mean(test_acc))

100%|██████████| 6/6 [00:02<00:00,  2.17it/s]

Testing Accuracy:  0.9266666666666667
